# Segmenting and Clustering Neighborhoods in Toronto, Canada
By: Kevin Moy

# Table of Contents
1. [Section 1: Reading and Cleaning Wikipedia's Postal Codes](#s1)
    1. [Download Data](#s11)
    2. [Clean Data](#s12)
    3. [Result and Assumptions](#s13)
2. [Section 2: Latitudes and Longitudes of Neighborhoods](#s2)
    1. [The Goal, Import Data](#s21)
    2. [Resulting Data Frame](#s22)
3. [Section 3: Explore and Cluster Neighborhoods in Toronto](#s3)
    1. [Goal and Credentials](#s31)
    2. [Practice FourSquare API Call and Narrow Search](#s32)
    3. [Processing API Call Data](#s33)
    4. [Analysis: Transforming Venue Types by Neighborhood](#s34)
    5. [Analysis: Most Common Venue Types by Neighborhood](#s35)
    6. [Clustering Neighborhoods](#s36)
    7. [Analysis: Cluster Analysis](#s37)


## Section 1: Reading and Cleaning Wikipedia's Postal Codes into Python <a name="s1">

In [1]:
# Download necessary packages
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

### 1.1 Download data <a name="s11"></a>

In [2]:
# Source: https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722
# Use the requests library to ping the website, return HTML.
website_html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# We use BeautifulSoup to parse HTML / XML documents to create a parse tree
!pip install beautifulsoup4-4.9.0-py3-none-any.whl
from bs4 import BeautifulSoup

Processing ./beautifulsoup4-4.9.0-py3-none-any.whl


In [3]:
soup = BeautifulSoup(website_html, 'html')
print(soup.prettify()[0:12000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptG7wpAMNAAAUaw2@sAAAES","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [4]:
My_table = soup.find('table',{'class':'wikitable'})

# First we need our table headers, marked by th
table_headers = My_table.findAll('th')
# Find the strings of results, kill new line
table_headers = [table_header.string for table_header in table_headers]
table_headers = [table_header.replace('\n','') for table_header in table_headers]
print('The table headers are: {}'.format(table_headers))

# Second we need table contents, each row marked by tr
# Load each row into a dictionary, to load into table_dict
# That table_dict will become the dataframe, with titles table_headers
table_contents = My_table.findAll('tr')[1:] # not header
table_dictionary = []
for row in table_contents:
    dict_row = {}
    for num, cell in enumerate(row.findAll('td')):
        dict_row.update({f'{table_headers[num]}':str(cell.string).replace('\n','')})
    table_dictionary.append(dict_row)
print('table dictionary is',table_dictionary[0:10])


The table headers are: ['Postal code', 'Borough', 'Neighborhood']
table dictionary is [{'Postal code': 'M1A', 'Borough': 'Not assigned', 'Neighborhood': ''}, {'Postal code': 'M2A', 'Borough': 'Not assigned', 'Neighborhood': ''}, {'Postal code': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'Postal code': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'Postal code': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park / Harbourfront'}, {'Postal code': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor / Lawrence Heights'}, {'Postal code': 'M7A', 'Borough': 'Downtown Toronto', 'Neighborhood': "Queen's Park / Ontario Provincial Government"}, {'Postal code': 'M8A', 'Borough': 'Not assigned', 'Neighborhood': ''}, {'Postal code': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'Postal code': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern / Rouge'}]


In [5]:
# Let's convert these rows of dictionaries into our beloved pandas dataframe
toronto_data = pd.DataFrame(table_dictionary, columns=table_headers)
toronto_data.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### 1.2 Clean data in listed ways: <a name="s12"></a>

3. To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [6]:
# Ignore boroughs not assigned
toronto_data_mod = toronto_data[toronto_data['Borough']!='Not assigned']
postal_codes = list(set(toronto_data_mod['Postal code']))


In [7]:
# For every postal code area, combine neighborhoods and boroughs
# For neighborhoods not assigned, name the borough instead.
boroughs, neighborhoods = [], []
for postal_code in postal_codes:
    boroughs.append(list(toronto_data_mod[toronto_data_mod['Postal code'] == postal_code]['Borough'].unique()))
    if list(toronto_data_mod[toronto_data_mod['Postal code'] == postal_code]) == []:
        neighborhoods.append(boroughs[-1])
    else:
        neighborhoods.append(list(set(toronto_data_mod[toronto_data_mod['Postal code'] == postal_code]['Neighborhood'])))
boroughs = [''.join(borough) for borough in boroughs]
neighborhoods = [', '.join(neighborhood) for neighborhood in neighborhoods]

In [8]:
toronto_data_mod = pd.DataFrame({'Postal code':postal_codes, 'Borough':boroughs, 'Neighborhood':neighborhoods})


toronto_data_mod.head()

,Postal code,Borough,Neighborhood
0,M9W,Etobicoke,Northwest
1,M5S,Downtown Toronto,University of Toronto / Harbord
2,M3J,North York,Northwood Park / York University
3,M2H,North York,Hillcrest Village
4,M9C,Etobicoke,Eringate / Bloordale Gardens / Old Burnhamthor...


### 1.3 Result and Assumptions <a name="s13"></a>

In [9]:
# Print the shape of the dataset
print('The size of the dataset is',toronto_data_mod.shape)



The size of the dataset is (103, 3)


Section 1 Assumptions and Observations

Assumptions:
All HTML code for the Wikipedia table is the same (untrue observation explains why I get None for MV5 Postal code when requested).


## Section 2: Latitudes and Longitudes of Neighborhoods <a name="s2"></a>

### 2.1 The Goal, Import Data: <a name="s21"></a>
We want to find the Latitudes and Longitudes of Neighborhoods given the Geospatial Coordinates file provided by IBM. It has Lat and Lon values given a Zip Code.

In [10]:
# import Geospatial_Coordinates.csv
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# Merge Geospatial Coordinates with our data table
toronto_data_mod.columns = ['Postal Code','Borough','Neighborhood']
toronto_df = toronto_data_mod.join(geo_df.set_index('Postal Code'),on='Postal Code')
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,M5S,Downtown Toronto,University of Toronto / Harbord,43.662696,-79.400049
2,M3J,North York,Northwood Park / York University,43.767980,-79.487262
3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,M9C,Etobicoke,Eringate / Bloordale Gardens / Old Burnhamthor...,43.643515,-79.577201


### 2.2: Print the DataFrame <a name="s22"></a>

In [12]:
# Print the entire dataframe
toronto_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,M5S,Downtown Toronto,University of Toronto / Harbord,43.662696,-79.400049
2,M3J,North York,Northwood Park / York University,43.767980,-79.487262
3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,M9C,Etobicoke,Eringate / Bloordale Gardens / Old Burnhamthor...,43.643515,-79.577201
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
6,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
7,M2N,North York,Willowdale,43.770120,-79.408493
8,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
9,M2R,North York,Willowdale,43.782736,-79.442259


## Section 3: Explore and Cluster Neighborhoods in Toronto <a name="s3"></a>
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

### 3.1 Goal and Credentials: <a name="s31"></a>
Cluster Toronto's neighborhoods and see which cluster and neighborhoods best exemplify Toronto's diverse food scene.

In [38]:
# Foursquare Credentials

print('Foursquare Credentials found!')

Foursquare Credentials found!


### 3.2 Practice FourSquare API Call and Narrow Search <a name = "s32"></a>

In [14]:
# Get top 100 venues in Northwest within 500 meters, first row in toronto_df
neighborhood_latitude = toronto_df['Latitude'][0]
neighborhood_longitude = toronto_df['Longitude'][0]
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?ll={},{}&client_id={}&client_secret={}&radius={}&v={}&limit={}'.format(neighborhood_latitude,neighborhood_longitude,CLIENT_ID,CLIENT_SECRET,radius,VERSION,LIMIT)


results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea771ca60ba08001b35f0b3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.711248304499996,
    'lng': -79.58784096674265},
   'sw': {'lat': 43.70224829549999, 'lng': -79.60026783325736}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5273f6f311d2137549494e29',
       'name': 'Economy Rent A Car',
       'location': {'lat': 43.708471,
        'lng': -79.589943,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.708471,
          'lng': -79.589943}],


In [15]:
# Practice getting categories. 
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5273f6f311d2137549494e29',
   'name': 'Economy Rent A Car',
   'location': {'lat': 43.708471,
    'lng': -79.589943,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.708471,
      'lng': -79.589943}],
    'distance': 382,
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddress': ['Canada']},
   'categories': [{'id': '4bf58dd8d48988d1ef941735',
     'name': 'Rental Car Location',
     'pluralName': 'Rental Car Locations',
     'shortName': 'Rental Car',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/airport_rentalcar_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-5273f6f311d2137549494e29-0'},
 {'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'global

### Popular Toronto Locations
We notice that this location outside of Toronto has only two locations. Let's limit our search to Boroughs that contain the word Toronto

In [16]:
# Filter toronto_df to toronto boroughs.
#print(toronto_df[['Borough']].values)
#print(['Toronto' in borough[0] for borough in toronto_df[['Borough']].values])
toronto_df = toronto_df[['Toronto' in borough[0] for borough in toronto_df[['Borough']].values]].reset_index(drop=True)
toronto_df.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5S,Downtown Toronto,University of Toronto / Harbord,43.662696,-79.400049
1,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049
2,M6R,West Toronto,Parkdale / Roncesvalles,43.648960,-79.456325
3,M5R,Central Toronto,The Annex / North Midtown / Yorkville,43.672710,-79.405678
4,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
8,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420
9,M4S,Central Toronto,Davisville,43.704324,-79.388790


### 3.3 Processing API Call Data <a name="s33"></a>

In [17]:
# Define a function to get categories of each venue fed
def get_category_type(row):
    try: 
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
        


In [18]:
# Let's practice grabbing venues from the first neighborhood.
from pandas import json_normalize

neighborhood_latitude = toronto_df['Latitude'][0]
neighborhood_longitude = toronto_df['Longitude'][0]
url = 'https://api.foursquare.com/v2/venues/explore?ll={},{}&client_id={}&client_secret={}&radius={}&v={}&limit={}'.format(neighborhood_latitude,neighborhood_longitude,CLIENT_ID,CLIENT_SECRET,radius,VERSION,LIMIT)


results = requests.get(url).json()
results

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues[filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Rasa,Restaurant,43.662757,-79.403988
2,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746
3,Piano Piano,Italian Restaurant,43.662949,-79.402898
4,Her Father's Cider Bar + Kitchen,Beer Bar,43.662448,-79.404703


In [19]:
# Better headers for our beloved dataframe
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]
print(f'There are {nearby_venues.shape[0]} venues.')
nearby_venues.head()

There are 34 venues.


,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Rasa,Restaurant,43.662757,-79.403988
2,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746
3,Piano Piano,Italian Restaurant,43.662949,-79.402898
4,Her Father's Cider Bar + Kitchen,Beer Bar,43.662448,-79.404703


In [20]:
# With that proof of concept, repeat that function for all of Toronto.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # API call
        url = 'https://api.foursquare.com/v2/venues/explore?ll={},{}&client_id={}&client_secret={}&radius={}&v={}&limit={}'.format(lat,lng,CLIENT_ID,CLIENT_SECRET,radius,VERSION,LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # Return relevant venue info
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood',
                                'Neighborhood Latitude',
                                'Neighborhood Longitude',
                                'Venue',
                                'Venue Latitude',
                                'Venue Longitude',
                                'Venue Category']
    return(nearby_venues)

In [21]:
# Create a dataframe of neighborhoods and associated venues
toronto_venues = getNearbyVenues(toronto_df['Neighborhood'],
                                toronto_df['Latitude'],
                                toronto_df['Longitude'])
print(toronto_venues.shape)


University of Toronto / Harbord
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Parkdale / Roncesvalles
The Annex / North Midtown / Yorkville
Forest Hill North & West
Berczy Park
North Toronto West
St. James Town
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Davisville
Lawrence Park
Toronto Dominion Centre / Design Exchange
Moore Park / Summerhill East
India Bazaar / The Beaches West
Business reply mail Processing CentrE
Dufferin / Dovercourt Village
Commerce Court / Victoria Hotel
Church and Wellesley
Queen's Park / Ontario Provincial Government
The Danforth West / Riverdale
Runnymede / Swansea
Rosedale
Regent Park / Harbourfront
Garden District, Ryerson
Davisville North
Studio District
The Beaches
Stn A PO Boxes
Harbourfront East / Union Station / Toronto Islands
Kensington Market / Chinatown / Grange Park
St. James Town / Cabbagetown
First Canadian Place / Underground city
Brockton / Parkdale Vi

In [22]:
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5S,Downtown Toronto,University of Toronto / Harbord,43.662696,-79.400049
1,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049
2,M6R,West Toronto,Parkdale / Roncesvalles,43.648960,-79.456325
3,M5R,Central Toronto,The Annex / North Midtown / Yorkville,43.672710,-79.405678
4,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307


### 3.4 Analysis: Transforming Venue Types by Neighborhood <a name="s34"></a>

In [23]:
# One hot encoding for Toronto Venue Categories
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix='',prefix_sep='')
# Add neighborhood
toronto_onehot['Neighborhood']=toronto_venues['Neighborhood']
toronto_onehot.head()


# Neighborhood as first column
n = list(toronto_onehot.columns).index("Neighborhood")
ideal_columns = [toronto_onehot.columns[n]]+list(toronto_onehot.columns[0:n])+list(toronto_onehot.columns[n+1:])
#print(ideal_columns)
toronto_onehot = toronto_onehot[ideal_columns]
toronto_onehot.head(10)


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,University of Toronto / Harbord,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,University of Toronto / Harbord,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
# Turn dummy variables into dummy densities within neighborhoods
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.017241,0.034483,0.000000,0.000000,0.00,0.017241,0.017241,0.000000,0.034483,0.000000,0.000000,0.017241,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.017241,0.051724,0.068966,0.000000,0.000,0.000000,0.00,0.000000,0.017241,0.000000,0.017241,0.000000,0.00,0.017241,0.000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.00,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.01724

### 3.5 Analysis: Most Common Venue Types by Neighborhood <a name="s35"></a>

In [25]:
# Print each neighborhood with top 5 common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print('----'+hood+'----')
    temp = toronto_grouped[toronto_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:] # remove Neighborhood line after Transposing
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2        Bakery  0.03
3      Beer Bar  0.03
4    Restaurant  0.03


----Brockton / Parkdale Village / Exhibition Place----
                   venue  freq
0                   Café  0.12
1            Coffee Shop  0.08
2         Breakfast Spot  0.08
3              Nightclub  0.08
4  Performing Arts Venue  0.04


----Business reply mail Processing CentrE----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2              Garden  0.06
3                Park  0.06
4          Comic Shop  0.06


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1  Italian Restaurant  0.

In [26]:
# Write a function to sort venues into descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
# In a new dataframe, display top 10 venues in each neighborhood
num_top_venues = 10
indicators = ['st','nd','rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# New Dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood']=toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:]= return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Italian Restaurant,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Café
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Nightclub,Performing Arts Venue,Bar,Music Venue,Burrito Place,Grocery Store,Gym
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Park,Pizza Place,Burrito Place,Restaurant,Brewery,Smoke Shop
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Airport,Harbor / Marina,Coffee Shop,Sculpture Garden,Boutique,Bar,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Salad Place,Fried Chicken Joint,Restaurant


### 3.6 Clustering Neighborhoods <a name="s36"></a>

In [28]:
# Cluster these neighborhoods! Use the frequencies dataframe
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',axis=1)
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 0, 1, 0,
       0, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [29]:
# Insert kmeans labels into sorted venues dataframe
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)


In [30]:
# Create a neighborhood dataframe with top 10 venues
toronto_merged = toronto_df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5S,Downtown Toronto,University of Toronto / Harbord,43.662696,-79.400049,0,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Bookstore,Bar,Bakery,Chinese Restaurant,Sandwich Place,Comfort Food Restaurant
1,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049,0,Coffee Shop,Pub,Supermarket,Bank,Sports Bar,Sushi Restaurant,Pizza Place,Liquor Store,Fried Chicken Joint,Light Rail Station
2,M6R,West Toronto,Parkdale / Roncesvalles,43.648960,-79.456325,0,Gift Shop,Movie Theater,Dessert Shop,Bookstore,Bar,Cuban Restaurant,Dog Run,Restaurant,Italian Restaurant,Coffee Shop
3,M5R,Central Toronto,The Annex / North Midtown / Yorkville,43.672710,-79.405678,0,Sandwich Place,Café,Coffee Shop,Donut Shop,Pharmacy,Indian Restaurant,Flower Shop,History Museum,BBQ Joint,Middle Eastern Restaurant
4,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [31]:
toronto_data.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [32]:
# Visualize the clusters! 

latitude, longitude = 43.6532, -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# color scheme
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map corresponding to color scheme
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
map_clusters

### 3.7 Cluster Analysis <a name="s37"></a>

### Cluster 0 Analysis:
Cluster 0 seems to have coffee shops, quick meals, and shopping.

In [33]:
# Let's do some analysis of the clusters! Cluster 0
toronto_merged.loc[toronto_merged['Cluster Labels']==0, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Bookstore,Bar,Bakery,Chinese Restaurant,Sandwich Place,Comfort Food Restaurant
1,Central Toronto,0,Coffee Shop,Pub,Supermarket,Bank,Sports Bar,Sushi Restaurant,Pizza Place,Liquor Store,Fried Chicken Joint,Light Rail Station
2,West Toronto,0,Gift Shop,Movie Theater,Dessert Shop,Bookstore,Bar,Cuban Restaurant,Dog Run,Restaurant,Italian Restaurant,Coffee Shop
3,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Donut Shop,Pharmacy,Indian Restaurant,Flower Shop,History Museum,BBQ Joint,Middle Eastern Restaurant
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Italian Restaurant,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Café
6,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Bagel Shop,Mexican Restaurant,Rental Car Location,Spa,Restaurant,Chinese Restaurant,Toy / Game Store
7,Downtown Toronto,0,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Italian Restaurant,Clothing Store,Department Store,Seafood Restaurant,Moroccan Restaurant
8,Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Airport,Harbor / Marina,Coffee Shop,Sculpture Garden,Boutique,Bar,Boat or Ferry
9,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Café,Gym,Coffee Shop,Italian Restaurant,Sushi Restaurant,Deli / Bodega,Bar
11,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,Salad Place,Deli / Bodega,American Restaurant,Asian Restaurant


### Cluster 1 Analysis
Cluster 1 differs because it has the biggest garden density. These are popular with tourists.

In [34]:
# Let's do some analysis of the clusters! Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels']==1, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Central Toronto,1,Playground,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


### Cluster 2 Analysis
Cluster 2 differentiates itself with the best playgrounds and general non-ethnic restaurants.

In [35]:
# Let's do some analysis of the clusters! Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels']==2, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Central Toronto,2,Garden,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


### Cluster 3 Analysis
Cluster 3 features natural activities, with its trails, parks, playgrounds, and dog runs. Athletic people go here!

In [36]:
# Let's do some analysis of the clusters! Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels']==3, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster 4 Analysis
Cluster 4 may be known for its pub-goers, who spend the next day going to the yoga studio, trail, and health food store. This is the most healthy part of town!

In [37]:
# Let's do some analysis of the clusters! Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels']==4, toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Central Toronto,4,Park,Bus Line,Swim School,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
21,Downtown Toronto,4,Park,Trail,Playground,Yoga Studio,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
